In [ ]:
# Set up a dictionary of signs
signs = dict()
signs['union'] = 'v'
signs['intersection'] = '^'
signs['implication'] = '->'
signs['negation'] = '~'

# Set up list of atomic statements
atomic = []
atomic.add('A')
atomic.add('B')
atomic.add('C')
atomic.add('D')

# Test input
input = '->->A->BC->->AB->AC'

In [ ]:
def alpha_beta(text):
    text = text[1:]
    alpha = ''
    beta = ''
    S1 = ''
    S2 = ''
    for char in text:
        if text[0] in signs.values():
            alpha += text[0]
            text = text[1:]
        elif text[0] in atomic:
            if text[1] is ',':
                S1 += text[:2]
                text = text[2:]
            else:
                alpha += text[0]
                text = text[1:]
        else:
            S1 += text[0]
            text = text[1:]

In [ ]:
def union(text):
    a = 1

In [ ]:
def neg_intersection(text):
    a = 1

In [ ]:
def implication(text):
    a = 1